# Description

This notebook creates a deep neural network model (MLP) to predict future returns based on fundamental factors.

I got the data from the SimFin free tier. I believe they give about 5 years of historical data for US
stocks and it is not quite recent

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sns
# Import the main functionality from the SimFin Python API.
import simfin as sf
# Import names used for easy access to SimFin's data-columns.
from simfin.names import *

In [2]:
# SimFin data-directory.
sf.set_data_dir('~/.simfin/data/')
# SimFin load API key or use free data.
sf.load_api_key(path='~/.simfin/api-key.txt', default_key='free')

In [3]:
# Seaborn set plotting style.
sns.set_style("whitegrid")

## Load Data

In [4]:
%%time
# Data for USA.
market = 'us'
# TTM Income Statements.
df_income_ttm = sf.load_income(variant='ttm', market=market)
df_income_qrt = sf.load_income(variant='quarterly', market=market)
# Quarterly Income Statements.
df_balance_ttm = sf.load_balance(variant='ttm', market=market)
df_balance_qrt = sf.load_balance(variant='quarterly', market=market)
# Quarterly Balance Sheets.
df_cashflow_ttm = sf.load_cashflow(variant='ttm', market=market)
df_cashflow_qrt = sf.load_cashflow(variant='quarterly', market=market)
# Daily Share-Prices.
df_prices = sf.load_shareprices(variant='daily', market=market)

Dataset "us-income-ttm" on disk (7 days old).
- Loading from disk ... 

/Users/marcusprewarski/projects/venv/lib/python3.11/site-packages/simfin/load.py:154: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df = pd.read_csv(path, sep=';', header=0,


Done!
Dataset "us-income-quarterly" on disk (6 days old).
- Loading from disk ... 

/Users/marcusprewarski/projects/venv/lib/python3.11/site-packages/simfin/load.py:154: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df = pd.read_csv(path, sep=';', header=0,


Done!
Dataset "us-balance-ttm" on disk (7 days old).
- Loading from disk ... 

...

Done!
Dataset "us-cashflow-quarterly" on disk (5 days old).
- Loading from disk ... 

/Users/marcusprewarski/projects/venv/lib/python3.11/site-packages/simfin/load.py:154: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df = pd.read_csv(path, sep=';', header=0,


Done!
Dataset "us-shareprices-daily" on disk (7 days old).
- Loading from disk ... 

/Users/marcusprewarski/projects/venv/lib/python3.11/site-packages/simfin/load.py:154: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df = pd.read_csv(path, sep=';', header=0,


Done!
CPU times: user 11.2 s, sys: 797 ms, total: 12 s
Wall time: 12.3 s


### Filter tickers
First filter out all tickers that are not in all datasets

Next lets get rid of all tickers that have a price in the datasets below some small value.
It might be better to limit by market cap but this might work well enough

In [5]:
df_prices = df_prices.rename(columns={ADJ_CLOSE: 'AdjClose'})

In [7]:
df_prices = df_prices.query('AdjClose > 0.25')
df_prices.shape

(5208144, 9)

In [13]:

p_ticks = set(df_prices.index.get_level_values(0))
i1_ticks = set(df_income_ttm.index.get_level_values(0))
i2_ticks = set(df_income_qrt.index.get_level_values(0))
b1_ticks = set(df_balance_ttm.index.get_level_values(0))
b2_ticks = set(df_balance_qrt.index.get_level_values(0))
c1_ticks = set(df_cashflow_ttm.index.get_level_values(0))
c2_ticks = set(df_cashflow_qrt.index.get_level_values(0))

all_ticks = p_ticks.intersection(i1_ticks, i2_ticks, b1_ticks, b2_ticks, c1_ticks, c2_ticks)
tickers = list(all_ticks)
len(tickers)

3640

In [14]:
df_income_ttm = df_income_ttm.loc[tickers].copy()
df_income_qrt = df_income_qrt.loc[tickers].copy()
df_balance_ttm = df_balance_ttm.loc[tickers].copy()
df_balance_qrt = df_balance_qrt.loc[tickers].copy()
df_cashflow_ttm = df_cashflow_ttm.loc[tickers].copy()
df_cashflow_qrt = df_cashflow_qrt.loc[tickers].copy()
df_prices = df_prices.loc[tickers].copy()

## Calculate Signals

In [15]:
df_fin_signals = sf.fin_signals(df_prices=df_prices,
                                df_income_ttm=df_income_ttm,
                                df_balance_ttm=df_balance_ttm,
                                df_cashflow_ttm=df_cashflow_ttm,
                                fill_method='ffill')


df_val_signals = sf.val_signals(df_prices=df_prices,
                                df_income_ttm=df_income_ttm,
                                df_balance_ttm=df_balance_ttm,
                                df_cashflow_ttm=df_cashflow_ttm)

df_growth_signals = \
    sf.growth_signals(df_prices=df_prices,
                      df_income_ttm=df_income_ttm,
                      df_income_qrt=df_income_qrt,
                      df_balance_ttm=df_balance_ttm,
                      df_balance_qrt=df_balance_qrt,
                      df_cashflow_ttm=df_cashflow_ttm,
                      df_cashflow_qrt=df_cashflow_qrt)



/Users/marcusprewarski/projects/venv/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/marcusprewarski/projects/venv/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/marcusprewarski/projects/venv/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/marcusprewarski/projects/venv/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/marcusprewarski/projects/venv/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwarg

## Create DF of factors we want

In [16]:
df_factors = pd.DataFrame(index=df_prices.index)
df_factors['AdjClose'] = df_prices['AdjClose']
df_factors["Return 3"] = sf.rel_change(df=df_prices['AdjClose'], freq='bdays', bdays=63, future=True)
df_factors["Return 6"] = sf.rel_change(df=df_prices['AdjClose'], freq='bdays', bdays=126, future=True)
df_factors["Return 12"] = sf.rel_change(df=df_prices['AdjClose'], freq='bdays', bdays=252, future=True)
df_factors.describe()

,AdjClose,Return 3,Return 6,Return 12
count,3.715422e+06,3.487405e+06,3.261052e+06,2.813830e+06
mean,2.378708e+02,3.186336e-02,6.737002e-02,1.647150e-01
std,8.041123e+03,4.408853e-01,6.708640e-01,1.229736e+00
min,2.600000e-01,-9.990000e-01,-9.994836e-01,-9.989633e-01
25%,9.240000e+00,-1.472910e-01,-2.133484e-01,-3.079190e-01
50%,2.307000e+01,1.013171e-03,1.128987e-03,-4.062358e-04
75%,5.847000e+01,1.431353e-01,2.121212e-01,3.361234e-01
max,1.587600e+06,5.351724e+01,8.765306e+01,1.246889e+02


In [17]:
df_growth_wins = sf.winsorize(df=df_growth_signals, quantile=0.01)
df_fin_wins = sf.winsorize(df=df_fin_signals, quantile=0.01)
df_value_wins = sf.winsorize(df=df_val_signals, quantile=0.01)

In [18]:
df_factors[EARNINGS_GROWTH_QOQ] =  df_growth_wins[EARNINGS_GROWTH_QOQ]
df_factors[SALES_GROWTH_QOQ] =  df_growth_wins[SALES_GROWTH_QOQ]

df_factors[CURRENT_RATIO] =  df_fin_wins[CURRENT_RATIO]
df_factors[DEBT_RATIO] =  df_fin_wins[DEBT_RATIO]
df_factors[GROSS_PROFIT_MARGIN] =  df_fin_wins[GROSS_PROFIT_MARGIN]
df_factors[NET_PROFIT_MARGIN] =  df_fin_wins[NET_PROFIT_MARGIN]
df_factors[BUYBACK_RATIO] =  df_fin_wins[BUYBACK_RATIO]
df_factors[ROA] =  df_fin_wins[ROA]
df_factors[ROE] =  df_fin_wins[ROE]

df_factors[MARKET_CAP] =  df_value_wins[MARKET_CAP]
df_factors[PE] =  df_value_wins[PE]
df_factors[PFCF] =  df_value_wins[PFCF]
df_factors[PSALES] =  df_value_wins[PSALES]
df_factors[PRICE_BOOK] =  df_value_wins[PRICE_BOOK]

## Massage the data to be more ML friendly

In [22]:
df_factors.describe()

,AdjClose,Return 3,Return 6,Return 12,Earnings Growth QOQ,Sales Growth QOQ,Current Ratio,Debt Ratio,Gross Profit Margin,Net Profit Margin,Share Buyback / FCF,Return on Assets,Return on Equity,Market-Cap,P/E,P/FCF,P/Sales,Price to Book Value
count,1.726555e+06,1.726555e+06,1.726555e+06,1.726555e+06,1.726555e+06,1.726555e+06,1.726555e+06,1.726555e+06,1.726555e+06,1.726555e+06,1.726555e+06,1.726555e+06,1.726555e+06,1.726555e+06,1.726555e+06,1.726555e+06,1.726555e+06,1.726555e+06
mean,5.632307e+01,2.457287e-02,5.158134e-02,1.236829e-01,-1.202936e-01,4.111684e-02,2.508991e+00,3.320510e-01,4.268286e-01,-1.681880e-01,3.165777e-01,-1.815578e-02,-1.694145e-02,2.137117e+01,1.094936e+01,1.218271e+01,4.434769e+00,4.431461e+00
std,6.505443e+01,2.428971e-01,3.653501e-01,5.750646e-01,1.801426e+00,2.164378e-01,2.070461e+00,2.154548e-01,2.377891e-01,7.010802e-01,1.055455e+00,1.602342e-01,4.583977e-01,2.074324e+00,4.602368e+01,4.188652e+01,6.110786e+00,6.006821e+00
min,2.340000e+00,-4.290585e-01,-5.610708e-01,-7.221850e-01,-4.808897e+00,-4.214709e-01,4.998063e-01,1.721741e-02,4.706318e-02,-3.047107e+00,-2.157863e+00,-5.235230e-01,-1.431009e+00,1.723895e+01,-1.150737e+02,-9.474474e+01,1.703045e-01,-4.303670e+00
25%,1.239000e+01,-1.286549e-01,-1.900030e-01,-2.545829e-01,-7.058116e-01,-5.564435e-02,1.185285e+00,1.594612e-01,2.334284e-01,-7.734627e-02,0.000000e+00,-4.542085e-02,-1.071314e-01,1.992572e+01,-5.135855e+00,-3.243053e+00,8.027585e-01,1.286382e+00
50%,3.127000e+01,1.164144e-02,1.541522e-02,2.855713e-02,-9.585704e-02,2.552617e-02,1.818491e+00,3.159765e-01,3.949370e-01,3.706226e-02,4.191196e-02,2.541220e-02,6.906786e-02,2.148760e+01,1.256294e+01,1.091749e+01,1.971963e+00,2.530204e+00
75%,7.289000e+01,1.544460e-01,2.327434e-01,3.678213e-01,3.427562e-01,1.138775e-01,2.961384e+00,4.673797e-01,5.997297e-01,1.059749e-01,5.106196e-01,6.894598e-02,1.815572e-01,2.289672e+01,2.857545e+01,2.542574e+01,5.067450e+00,5.237665e+00
max,2.596700e+02,6.061870e-01,9.830666e-01,1.721371e+00,4.941176e+00,6.425321e-01,9.126078e+00,8.238911e-01,9.069016e-01,3.274140e-01,3.591735e+00,1.891279e-01,8.641251e-01,2.503662e+01,1.250420e+02,1.279845e+02,2.584951e+01,2.458208e+01


In [19]:
df_factors = df_factors.dropna()

In [20]:
df_factors[MARKET_CAP] = np.log(df_factors[MARKET_CAP])

In [21]:
df_factors = sf.winsorize(df=df_factors, quantile=0.04)

In [49]:
# at some point we lost a bunch of tickers, not sure where but for now lets get a new set
tickers = list(set(df_factors.index.get_level_values(0)))

In [50]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split

In [51]:
train_tickers, test_tickers = train_test_split(tickers, train_size=0.8, random_state=42)
len(train_tickers), len(test_tickers), len(tickers)

(1932, 483, 2415)

In [55]:
ts = pd.Timestamp('2019-01-02')

In [56]:
ts

Timestamp('2019-01-02 00:00:00')

In [77]:
ts = ts - pd.offsets.BusinessDay(n=20)
ts

Timestamp('2022-07-27 00:00:00')

In [82]:
ts < pd.Timestamp('2022-07-28')

True

## Create the train and test datasets

In [84]:
x_cols = [EARNINGS_GROWTH_QOQ, 
          SALES_GROWTH_QOQ,
         CURRENT_RATIO,
         DEBT_RATIO,
         GROSS_PROFIT_MARGIN,
         NET_PROFIT_MARGIN,
         BUYBACK_RATIO,
         ROA,
         ROE,
         MARKET_CAP,
         PE,
         PFCF,
         PSALES,
         PRICE_BOOK]

y_col = "Return 3"

In [87]:
def create_dataset(df, tickers, x_cols, y_col):
    df_subset = df.loc[tickers].copy()
    df_xs = df_subset[x_cols]
    df_ys = df_subset[y_col]
    return df_xs, df_ys

In [ ]:
X_train, y_train = create_dataset(df_factors, train_tickers, x_cols, y_col)
X_test, y_test = create_dataset(df_factors, test_tickers, x_cols, y_col)

## Scale the data

In [94]:
def scale_signal(df):
    """Apply the signal-scaler to the given DataFrame."""

    # Apply the scaler. This returns a numpy array.
    array = signal_scaler.transform(df)
    
    # Create a DataFrame with the correct column-names and index.
    df_scaled = pd.DataFrame(data=array,
                             columns=df.columns,
                             index=df.index)
    return df_scaled

In [93]:
signal_scaler = StandardScaler()
signal_scaler.fit(X_train)

StandardScaler()

In [95]:
X_train = scale_signal(X_train)
X_test = scale_signal(X_test)

## Build the Neural Network

In [108]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset

In [228]:
X_train_ten = torch.tensor(X_train.values, dtype=torch.float32)
y_train_ten = torch.tensor(y_train.values, dtype=torch.float32)
y_train_ten = torch.unsqueeze(y_train_ten, 1)
X_test_ten = torch.tensor(X_test.values, dtype=torch.float32)
y_test_ten = torch.tensor(y_test.values, dtype=torch.float32)
y_test_ten = torch.unsqueeze(y_test_ten, 1)

In [229]:
y_train_ten.shape

torch.Size([1395412, 1])

In [230]:
# Custom Dataset
class MyDataset(Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

In [231]:
train_ds = MyDataset(X_train_ten, y_train_ten)
test_ds = MyDataset(X_test_ten, y_test_ten)

In [232]:
train_loader = torch.utils.data.DataLoader(train_ds, batch_size=512, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_ds, batch_size=512, shuffle=False)

In [239]:
class MLP(nn.Module):
  '''
    Multilayer Perceptron.
  '''
  def __init__(self):
    super().__init__()
    self.layers = nn.Sequential(
      nn.Linear(14, 32),
      nn.ReLU(),
      nn.Linear(32, 16),
      nn.ReLU(),
      nn.Linear(16, 1)
    )


  def forward(self, x):
    '''Forward pass'''
    return self.layers(x)

In [240]:
torch.manual_seed(42)

In [241]:
# Initialize the MLP
mlp = MLP()
# Define the loss function and optimizer
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(mlp.parameters(), lr=1e-3)

In [242]:
# Run the training loop
for epoch in range(0, 10): # 5 epochs at maximum
    
    # Print epoch
    print(f'Starting epoch {epoch+1}')
    
    # Set current loss value
    current_loss = 0.0
    current_test_loss = 0.0
    
    mlp.train()
    # Iterate over the DataLoader for training data
    for i, data in enumerate(train_loader, 0):
      
        # Get inputs
        inputs, targets = data
      
        # Zero the gradients
        optimizer.zero_grad()
      
        # Perform forward pass
        outputs = mlp(inputs)

        #print(f'out shape {outputs.shape}')
        #print(f'targets shape {targets.shape}')
      
        # Compute loss
        loss = loss_function(outputs, targets)
      
        # Perform backward pass
        loss.backward()
      
        # Perform optimization
        optimizer.step()
      
        # Print statistics
        current_loss += loss.item()

    train_loss = current_loss / i
    print(f'{epoch=}, train loss {train_loss}')

    current_test_loss = 0.0
    mlp.eval()
    for i, data in enumerate(test_loader, 0):
        # Get inputs
        inputs, targets = data

        # Perform forward pass
        outputs = mlp(inputs)

        # Compute loss
        loss = loss_function(outputs, targets)
        current_test_loss += loss.item()

    test_loss = current_test_loss / i
    print(f'{epoch=}, test loss {test_loss}')

    


      
# Process is complete.
print('Training process has finished.')

Starting epoch 1
epoch=0, train loss 0.05737535695827335
epoch=0, test loss 0.05711236883289305
Starting epoch 2
epoch=1, train loss 0.05611503255066522
epoch=1, test loss 0.05745520958772085
Starting epoch 3
epoch=2, train loss 0.05544866596750163
epoch=2, test loss 0.0583243990366004
Starting epoch 4
epoch=3, train loss 0.05504066009723812
epoch=3, test loss 0.05843496419872869
Starting epoch 5
epoch=4, train loss 0.054782804756536393
epoch=4, test loss 0.05878835539665108
Starting epoch 6
epoch=5, train loss 0.05463810332051111
epoch=5, test loss 0.05900845287119733
Starting epoch 7
epoch=6, train loss 0.0544955742058404
epoch=6, test loss 0.0590932720907367
Starting epoch 8
epoch=7, train loss 0.05438000734929645
epoch=7, test loss 0.05926096222245195
Starting epoch 9
epoch=8, train loss 0.05426521712623605
epoch=8, test loss 0.0592813022894606
Starting epoch 10
epoch=9, train loss 0.05418656124160924
epoch=9, test loss 0.05926060552951006
Training process has finished.


# Conclusion
